In [ ]:
import datetime
!pip install python-dotenv

In [ ]:
!pip list

In [1]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
def fetch_data(url, api_key):
    headers = {'X-API-Key': api_key}
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # This will raise an exception for HTTP errors

    total_pages = response.headers.get('X-Total-Pages', 1)  # Default to 1 if the header is not found
    return response.json(), int(total_pages)

def extract_payload_and_time(data):
    return [(entry['payload'], entry['rx_metadata'][0].get('received_at')) for entry in data]


def main():
    base_url = os.getenv('BASE_URL')
    api_key = os.getenv('X-API-Key')      
    all_data = []
    
    # Fetch data from the first page to get the total number of pages
    first_page_url = base_url + '1'
    first_page_data, total_pages = fetch_data(first_page_url, api_key)
    all_data.extend(extract_payload_and_time(first_page_data))

    # Fetch data from the remaining pages
    for page in range(2, total_pages + 1):
        url = base_url + str(page)
        data, _ = fetch_data(url, api_key)
        all_data.extend(extract_payload_and_time(data))

    return all_data

In [4]:
load = True
if not load:
    all_data = main()
    all_data

In [5]:
import json

In [6]:
filename = 'globalsat_lt501e.json'

if not load:
    # Writing to a JSON file
    with open(filename, 'w') as file:
        json.dump(all_data, file, indent=4)
    
    print(f"Data has been exported to {filename}")
else:
    # Reading from a JSON file
    with open(filename) as file:
        all_data = json.load(file)

In [8]:
all_data[0]

['DBACkQOYAIDG9wITBekDTGU=', '2023-11-08T21:56:42.742723187Z']

In [9]:
import base64

# Example base64 encoded string
encoded_value = all_data[0][0]

# Decode the base64 encoded string
decoded_value = base64.b64decode(encoded_value)
decoded_value.hex()

'0c10029103980080c6f7021305e9034c65'

In [11]:
from parser.parser_globalsat_lt501e import LT501Parser

parser = LT501Parser(debug_level=99)
output = parser.parse_bytes_hex(decoded_value.hex())
output
print(output)
print(output['lat'])

{'version': 12, 'command_id': 4098, 'lon': 9.962385, 'lat': 49.79264, 'gps_fix': 0, 'report_type': 19, 'batt': 5, 'timestamp': 1699480553}
49.79264


In [36]:
import base64
from parser.parser_globalsat_lt501e import LT501Parser
from dateutil import parser as date_parser

# Initialize the parser
parser = LT501Parser(debug_level=99)

# List to hold the parsed data
parsed_data = []

for encoded_value, time in all_data:
    # Decode the base64 encoded string
    decoded_value = base64.b64decode(encoded_value)

    # Use the parser to parse the decoded data
    parsed_entry = parser.parse_bytes_hex(decoded_value.hex())
    if 'lon' in parsed_entry and 'lat' in parsed_entry:
        
        # Append the parsed data in the required format
        parsed_data.append({
            'coordinates': [parsed_entry['lon'], parsed_entry['lat']],
            'time': date_parser.isoparse(time)
        })
    else:
        print(f"Missing 'lon' or 'lat' in entry: {parsed_entry}")
        continue    
    
parsed_data

Missing 'lon' or 'lat' in entry: {'version': 12, 'command_id': 4866, 'beacon_id': '74278bdab64445208f0c720eaf05993500003e1c', 'beacon_type': 1, 'report_type': 2, 'rssi': 196, 'tx_power': 197, 'batt': 30}
Missing 'lon' or 'lat' in entry: {'version': 12, 'command_id': 4866, 'beacon_id': '74278bdab64445208f0c720eaf05993500007822', 'beacon_type': 1, 'report_type': 2, 'rssi': 165, 'tx_power': 197, 'batt': 55}


[{'coordinates': [9.962385, 49.79264],
  'time': datetime.datetime(2023, 11, 8, 21, 56, 42, 742723, tzinfo=tzutc())},
 {'coordinates': [9.962385, 49.79264],
  'time': datetime.datetime(2023, 11, 8, 21, 55, 54, 774553, tzinfo=tzutc())},
 {'coordinates': [9.962385, 49.79264],
  'time': datetime.datetime(2023, 11, 8, 21, 54, 54, 773257, tzinfo=tzutc())},
 {'coordinates': [9.962385, 49.79264],
  'time': datetime.datetime(2023, 11, 8, 21, 53, 54, 769403, tzinfo=tzutc())},
 {'coordinates': [9.962385, 49.79264],
  'time': datetime.datetime(2023, 11, 8, 21, 52, 54, 746879, tzinfo=tzutc())},
 {'coordinates': [9.962385, 49.79264],
  'time': datetime.datetime(2023, 11, 8, 21, 51, 54, 748187, tzinfo=tzutc())},
 {'coordinates': [9.962385, 49.79264],
  'time': datetime.datetime(2023, 11, 8, 21, 50, 54, 743505, tzinfo=tzutc())},
 {'coordinates': [9.962385, 49.79264],
  'time': datetime.datetime(2023, 11, 8, 21, 49, 54, 750380, tzinfo=tzutc())},
 {'coordinates': [9.962385, 49.79264],
  'time': datetim

In [ ]:
from map.plot_gps_history import create_geojson_features
import folium
from folium.plugins import TimestampedGeoJson

# Create map object
m = folium.Map(location=parsed_data[0]['coordinates'], zoom_start=3)

# Create features
features = create_geojson_features(parsed_data)

# Create a TimestampedGeoJson object and add it to the map
TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M',
    transition_time=1000,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY/MM/DD HH:mm:ss',
    time_slider_drag_update=True,
    add_last_point=True).add_to(m)

# Save to an HTML file
m.save('my_animated_map.html')

print("Map has been saved to my_animated_map.html")


In [247]:
import pydeck as pdk
import pandas as pd

original_data = parsed_data#[:10]

timestamps = [int(item['time'].timestamp()) for item in original_data]
coordinates = [item['coordinates'] for item in original_data]

cmp_coordinate = coordinates[0]
DELTA = 0.001

print(len(timestamps))

sorted_pairs = []
for time, coord in zip(timestamps, coordinates):
    sorted_pairs.append((time, coord))

sorted_pairs.sort(key=lambda pair: pair[0])
timestamps, coordinates = zip(*sorted_pairs)

# Converting tuples to lists
timestamps = list(timestamps)
coordinates = list(coordinates)

1068


In [249]:
filtered_timestamps = []
filtered_coordinates = []
cmp_coordinate = coordinates[0]

for timestamp, coordinate in zip(timestamps, coordinates):
    if not (abs(coordinate[0] - cmp_coordinate[0]) <= DELTA and abs(coordinate[1] - cmp_coordinate[1]) <= DELTA):
        filtered_timestamps.append(timestamp)
        filtered_coordinates.append(coordinate)
        cmp_coordinate = coordinate

timestamps, coordinates = filtered_timestamps, filtered_coordinates

print(len(timestamps))

# Since the data is for a single trip, we create a DataFrame with a single entry
df = pd.DataFrame({
    'coordinates': [coordinates],
    'timestamps': [timestamps]
})

df

55


,coordinates,timestamps
0,"[[9.96398, 49.792288], [9.954263, 49.794442], ...","[1699405619, 1699409219, 1699409759, 169941071..."


In [217]:
df['timestamps']

0    [1699480554, 1699480434, 1699480314, 169948019...
Name: timestamps, dtype: object

In [214]:
timestamps = df['timestamps'][0]
timestamps.sort(reverse=False)
min_timestamp = timestamps[0]
max_timestamp = timestamps[-1]

In [202]:
df['timestamps'] = df["timestamps"].apply(lambda f: [item - min_timestamp for item in f])
df

,coordinates,timestamps
0,"[[9.962385, 49.79264], [9.962385, 49.79264], [...","[0, 120, 180, 299, 360, 480, 540, 660, 720, 84..."


In [203]:
min(df['timestamps'][0])

0

In [204]:
# Create TripsLayer
layer = pdk.Layer(
    "TripsLayer",
    df,
    get_path='coordinates',
    get_timestamps='timestamps',
    get_color=[253, 128, 93],
    opacity=0.8,
    width_min_pixels=5,
    rounded=True,
    trail_length=600,
    current_time=10_000,
)

# Set the initial view state
view_state = pdk.ViewState(
    latitude=original_data[0]['coordinates'][1],
    longitude=original_data[0]['coordinates'][0],
    zoom=13,
    bearing=0,
    pitch=45
)

# Render the deck
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html('trips_layer.html')

## 1st try, not working

In [ ]:
import pydeck as pdk
import pandas as pd

# Convert to DataFrame
df = pd.DataFrame(parsed_data)

# Convert time to timestamps
df['time'] = pd.to_datetime(df['time'])
start_time = df['time'].min()
df['timestamps'] = (df['time'] - start_time).dt.total_seconds() * 1000

# Create a single path
path = [row['coordinates'] for _, row in df.iterrows()]
timestamps = df['timestamps'].tolist()

# Create TripsLayer
layer = pdk.Layer(
    "TripsLayer",
    {"path": path, "timestamps": timestamps},
    get_color=[253, 128, 93],
    opacity=0.8,
    width_min_pixels=5,
    rounded=True,
    trail_length=600,
    current_time=0,
)

# Set the initial view state
view_state = pdk.ViewState(
    latitude=df['coordinates'].iloc[0][1],
    longitude=df['coordinates'].iloc[0][0],
    zoom=11,
    bearing=0,
    pitch=45
)

# Render the deck
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html('trips_layer.html')

# Have a look at the example

In [138]:
"""
TripsLayer
==========

Plot of a single vehicle trip within San Francisco, fading in from the origin.

Adapted from a deck.gl documentation example.
"""

import pydeck as pdk
import pandas as pd

TRIPS_LAYER_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/sf.trips.json"  # noqa

df = pd.read_json(TRIPS_LAYER_DATA)
df

,waypoints
0,"[{'coordinates': [-122.39079879999997, 37.7664..."


In [139]:
df["coordinates"] = df["waypoints"].apply(lambda f: [item["coordinates"] for item in f])
df["timestamps"] = df["waypoints"].apply(lambda f: [item["timestamp"] - 1554772579000 for item in f])

df.drop(["waypoints"], axis=1, inplace=True)
df

,coordinates,timestamps
0,"[[-122.39079879999997, 37.7664413], [-122.3908...","[0, 9, 54, 92, 345, 402, 462, 563, 880, 1070, ..."


In [ ]:
layer = pdk.Layer(
    "TripsLayer",
    df,
    get_path="coordinates",
    get_timestamps="timestamps",
    get_color=[253, 128, 93],
    opacity=0.8,
    width_min_pixels=5,
    rounded=True,
    trail_length=600,
    current_time=500,
)

view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=11, bearing=0, pitch=45)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("example_trips_layer.html")